In [2]:
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import RidgeClassifier
from lightgbm import LGBMClassifier

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, auc
import numpy as np
from sklearn.model_selection import cross_val_score
import time as time
import joblib

In [12]:
df = pd.read_csv('postcovid.csv')
df.dtypes

Unnamed: 0      int64
location       object
As            float64
Cd            float64
Ni            float64
Fe            float64
Pb            float64
Zn            float64
dtype: object

In [13]:
df.columns[df.isna().any()]

Index([], dtype='object')

##### Label Encoding

In [14]:
df

,Unnamed: 0,location,As,Cd,Ni,Fe,Pb,Zn
0,0,ramganga_river,0.00167,0.029000,0.120000,1.900000,0.000000,1.420000
1,1,katghar_handpump,0.00254,0.049000,0.080000,0.320000,0.000000,0.632000
2,2,kunwar_saheb_lane,0.00392,0.078000,0.190000,1.200000,0.045000,2.700000
3,3,ramganga,0.00704,0.105000,0.090000,2.000000,0.589600,0.500000
4,4,near_ramganga_handpump,0.00905,0.132000,0.070000,1.200000,0.792000,1.550000
...,...,...,...,...,...,...,...,...
157,157,warsi_nagar21,0.01580,0.030359,0.072439,0.270531,0.010781,1.017052
158,158,warsi_nagar22,0.01590,0.029162,0.076563,0.312961,0.010431,0.989028
159,159,warsi_nagar23,0.01600,0.030312,0.072576,0.298951,0.010882,0.954478
160,160,warsi_nagar24,0.01610,0.032039,0.074604,0.274217,0.009732,1.031351


In [15]:
len(df['location'].unique())

162

In [16]:
df.shape

(162, 8)

In [17]:
df.drop(['Unnamed: 0','location'], axis=True, inplace=True)
df

,As,Cd,Ni,Fe,Pb,Zn
0,0.00167,0.029000,0.120000,1.900000,0.000000,1.420000
1,0.00254,0.049000,0.080000,0.320000,0.000000,0.632000
2,0.00392,0.078000,0.190000,1.200000,0.045000,2.700000
3,0.00704,0.105000,0.090000,2.000000,0.589600,0.500000
4,0.00905,0.132000,0.070000,1.200000,0.792000,1.550000
...,...,...,...,...,...,...
157,0.01580,0.030359,0.072439,0.270531,0.010781,1.017052
158,0.01590,0.029162,0.076563,0.312961,0.010431,0.989028
159,0.01600,0.030312,0.072576,0.298951,0.010882,0.954478
160,0.01610,0.032039,0.074604,0.274217,0.009732,1.031351


In [18]:
# df.drop(['S. No.','River','As','Toxicity'], axis=True, inplace=True)
# df

In [19]:
features = df
# labels = df['Toxicity']

In [20]:
features

,As,Cd,Ni,Fe,Pb,Zn
0,0.00167,0.029000,0.120000,1.900000,0.000000,1.420000
1,0.00254,0.049000,0.080000,0.320000,0.000000,0.632000
2,0.00392,0.078000,0.190000,1.200000,0.045000,2.700000
3,0.00704,0.105000,0.090000,2.000000,0.589600,0.500000
4,0.00905,0.132000,0.070000,1.200000,0.792000,1.550000
...,...,...,...,...,...,...
157,0.01580,0.030359,0.072439,0.270531,0.010781,1.017052
158,0.01590,0.029162,0.076563,0.312961,0.010431,0.989028
159,0.01600,0.030312,0.072576,0.298951,0.010882,0.954478
160,0.01610,0.032039,0.074604,0.274217,0.009732,1.031351


In [21]:
features.describe()

,As,Cd,Ni,Fe,Pb,Zn
count,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000
mean,0.008570,0.032517,0.072087,0.332911,0.023369,1.301433
std,0.004308,0.015767,0.014619,0.207758,0.097982,0.732317
min,0.001110,0.027056,0.063123,0.270215,0.000000,0.452611
25%,0.004825,0.028827,0.067286,0.288093,0.009550,0.639026
50%,0.008400,0.030275,0.070511,0.300198,0.009992,1.193060
75%,0.012175,0.031566,0.073489,0.316024,0.010498,1.571800
max,0.017440,0.178000,0.194000,2.000000,0.801000,2.964809


#### 1. Using LightGBM

In [36]:
start_time=time.time()
lgb_model = joblib.load('lgb_river_model')

y_pred = lgb_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
13.001680374145508 ms


###### 2. Using XGBOOST Classifier

In [37]:
start_time=time.time()
xgb_model = joblib.load('xgb_river_model')

y_pred = xgb_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
14.516592025756836 ms


###### 3. Using RandomForest Classifier

In [38]:
start_time=time.time()
rf_model = joblib.load('rf_river_model')

y_pred = rf_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
34.5151424407959 ms


#### 4. Using Decision Tree

In [39]:
start_time=time.time()
dt_model = joblib.load('dt_river_model')

y_pred = dt_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
5.000114440917969 ms


#### 5. Multinomial Naive Bayes

In [40]:
start_time=time.time()
mnb_model = joblib.load('mnb_river_model')

y_pred = mnb_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1
 0 1 0 1 1 1 0 1 0 1 1 1 0 0]
6.003141403198242 ms


#### 6. Gaussian Naive Bayes

In [41]:
start_time=time.time()
gnb_model = joblib.load('gnb_river_model')

y_pred = gnb_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
5.004644393920898 ms


#### 7. SVM

In [42]:
start_time=time.time()
svm_model = joblib.load('svm_river_model')

y_pred = svm_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
6.995916366577148 ms


#### 8. Ridge Classifier

In [43]:
start_time=time.time()
rg_model = joblib.load('rg_river_model')

y_pred = rg_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
6.008148193359375 ms


#### 9. Logistic Regression

In [45]:
start_time=time.time()
logr_model = joblib.load('logr_river_model')

y_pred = logr_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
5.5370330810546875 ms
